In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Layer, Conv2D, GlobalAveragePooling2D, Dense, Input
from keras.layers import MaxPool2D, BatchNormalization, Activation, Add, Dropout
import tensorflow_addons as tfa
tf.__version__

'2.10.1'

In [16]:
def SE_block(inputs, ratio=8):
    b, w, h, c = inputs.shape
    
    #* Squeeze
    x = GlobalAveragePooling2D()(inputs)
    
    #* Excitation - FC
    x = Dense(c// ratio, activation='relu', use_bias=False)(x)
    x = Dense(c, activation='sigmoid', use_bias=False)(x)
    
    #* scale
    x = tf.reshape(x, [-1, 1, 1, c])
    x = inputs * x 
    
    return x

In [37]:
#ct_model 

def model_block (inputs, n_filter=32, kernel_size=5):
    conv = Conv2D(n_filter, kernel_size, 1, padding='same')(inputs)
    mish1 = tfa.activations.mish(conv)
    max_pool = tf.keras.layers.MaxPool2D(2,2)(mish1)
    bn = tf.keras.layers.BatchNormalization()(max_pool)
    drp_out = Dropout(0.5)(bn)
    se = SE_block(drp_out)
    return se

#* builing a block 
n_classes = 3
input_shape = (256, 256, 3)
inputs = Input(shape=input_shape)
blck1 = model_block(inputs, 32)
blck2 = model_block(blck1, 64)
blck3 = model_block(blck2, 128)
blck4 = model_block(blck3, 256)

conv_last = Conv2D(256, 5, 1, padding='same')(blck4)
#fc1 = Dense(2048, activation='relu')(GlobalAveragePooling2D()(conv_last))
fc1 = Dense(65536, activation='relu')(GlobalAveragePooling2D()(conv_last))
fc2 = Dense(256*n_classes, activation='relu')(fc1)
output_layer = Dense(n_classes, activation='sigmoid')(fc2)

model = tf.keras.Model(inputs=inputs, outputs=output_layer)
model.summary()




Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_50 (Conv2D)             (None, 256, 256, 32  2432        ['input_20[0][0]']               
                                )                                                                 
                                                                                                  
 tf.convert_to_tensor_38 (TFOpL  (None, 256, 256, 32  0          ['conv2d_50[0][0]']              
 ambda)                         )                                                          

In [36]:
optimizer = tfa.optimizers.RectifiedAdam(
    lr = 0.01,
    beta_1=0.9, #* momentum
    weight_decay=1e-3, 
    epsilon=1e-10,
)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)#* softmax

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])


c:\Users\DELL\anaconda3\envs\tfgpy\lib\site-packages\tensorflow_addons\optimizers\rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


NameError: name 'SE_block' is not defined